In [1]:
import sys
sys.path.append('../..')
from config import client

from pymongo import UpdateOne
from tqdm import tqdm

from mlpp.data_collection.sample import ScoresSubset

In [2]:
osu_db = client['osu_random_db']

In [3]:
sample_3k = ScoresSubset(osu_db['scores_sample_3k'], osu_db['users_sample_3k'])

In [4]:
bms = list(osu_db['osu_beatmaps'].find({}, {'_id': 1}).sort('_id'))

In [22]:
updates = []
for i in range(len(bms)):
    updates.append(
        UpdateOne(
            {
                '_id': bms[i]['_id']
            }
            ,{
                '$set': {
                    'mlpp': {
                        'order': i
                    }
                }
            }
        )
    )

In [25]:
osu_db['osu_beatmaps'].bulk_write(updates)

In [5]:
bm_order = list(osu_db['osu_beatmaps'].find({}, {'mlpp.order': 1}))

In [6]:
bm_order = {bm['_id']: bm['mlpp']['order'] for bm in bm_order}

In [7]:
def compute_acc(score):
    c50, c100, c300, cmiss = score['count50'], score['count100'], score['count300'], score['countmiss']
    return (c50 / 6 + c100 / 3 + c300) / (c50 + c100 + c300 + cmiss)

In [8]:
def compare(s_i, s_j):
    return compute_acc(s_i) < compute_acc(s_j)

In [ ]:
fields = {
    'count50': 1,
    'count100': 1,
    'count300': 1,
    'countmiss': 1,
    'beatmap_id': 1
}

scores = list(sample_3k.scores_high.find({}, fields))
scores = sorted( for s in scores, key = la)

In [46]:
def beatmap_comparisons(user_id):
    
    ordered_scores = list(sample_3k.scores_high.find({'user_id': user_id}, fields).sort('date'))
    comparisons = []
    
    for i in range(len(ordered_scores) - 1):
        s_i, s_j = ordered_scores[i], ordered_scores[i + 1]
        
        to_order = lambda s: bm_order[s['beatmap_id']]
        i, j = to_order(s_i), to_order(s_j)
        
        obsv = [i, j] if compare(s_i, s_j) else [j, i]
        comparisons.append(obsv)
    
    return comparisons

In [38]:
user_ids = [u['_id'] for u in sample_3k.user_stats.find({}, {'_id': 1})]

In [52]:
comps = []
for user_id in tqdm(user_ids):
    comps.extend(beatmap_comparisons(user_id))

 14%|█▍        | 426/3000 [02:45<16:36,  2.58it/s]


KeyError: 1426164